# Raspagem de Dados com Selenium

Raspar os dados do [ClimaTempo](https://www.climatempo.com.br/previsao-do-tempo/fim-de-semana/cidade/558/saopaulo-sp)

## Configuração do notebook

In [9]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from lib.database import Database
from selenium import webdriver
from datetime import datetime
import pandas as pd
import json

DB_NAME = 'db.sqlite3'

# Funções auxiliares
def date_from_timestamp(timestamp):
    return timestamp.strftime('%Y-%m-%d')

def today():
    return datetime.today().strftime('%Y-%m-%d')

## Database Setup

In [2]:
db = Database(DB_NAME)
with db as connection:
    with open('data/locais.csv') as file:
        db.load_locais(file)

## Selenium Setup

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

[WDM] - Downloading: 100%|█████████████████| 6.05M/6.05M [00:00<00:00, 10.6MB/s]


OBS: o código abaixo está muito extenso e deve ser refatorado

In [4]:
with db as connection:
    # pega lista de locais
    locais = db.get_locais()
    dt_acesso = today()
    
    for local in locais:
        # acessa a URL do local
        driver.get(local['url'])
        
        print(f"Acesso no dia {dt_acesso} da cidade {local['nome']}:")
        for i in range(1,4):
            id = 'wrapper-chart-' + str(i)
            element = driver.find_element(By.ID, id)
            
            infos = json.loads(element.get_attribute('data-infos'))

            # registra o acesso no banco de dados
            dt_previsao = infos[0]['date'].split()[0]
            try:
                id_acesso = db.registra_acesso(local['id'], dt_acesso, dt_previsao)
                print(f"\tPrevisão para o dia {dt_previsao} feita!")
            except:
                print(f"\tJá existe previsão para o dia {dt_previsao}!")
                continue
                
            for obj in infos:
                db.registra_previsao(id_acesso, obj['date'].split()[1], obj['temperature']['temperature'])
                
driver.quit()

Acesso no dia 2022-08-28 da cidade São Paulo:
	Já existe previsão para o dia 2022-08-28!
	Já existe previsão para o dia 2022-08-29!
	Já existe previsão para o dia 2022-08-30!
Acesso no dia 2022-08-28 da cidade Salvador:
	Já existe previsão para o dia 2022-08-28!
	Já existe previsão para o dia 2022-08-29!
	Já existe previsão para o dia 2022-08-30!


# Exportando o banco para CSV

In [12]:
query = ''

with open('queries/all_previsoes.sql') as file:
    query = file.read()
    
with db as connection:
    
    result = db.con.cursor().execute(query).fetchall()